In [1]:
import numpy as np
import pickle
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K
import os

In [2]:
# === ✅ BASE PATH ===
BASE_PATH = "C:/Users/moksh/OneDrive/Documents/OneDrive/Desktop/LCIT/CV/Assignment_1"

# === ✅ LOAD DATA ===
X_img_train = np.load(os.path.join(BASE_PATH, "X_img_train_bal_q4.npy"), allow_pickle=True)
X_img_val = np.load(os.path.join(BASE_PATH, "X_img_val_bal_q4.npy"), allow_pickle=True)
y_train = np.load(os.path.join(BASE_PATH, "y_train_bal_q4.npy"))
y_val = np.load(os.path.join(BASE_PATH, "y_val_bal_q4.npy"))

with open(os.path.join(BASE_PATH, "metadata_seq.pkl"), "rb") as f:
    X_meta_full = pickle.load(f)

with open(os.path.join(BASE_PATH, "label_encoder.pkl"), "rb") as f:
    label_encoder = pickle.load(f)

X_meta_train = X_meta_full[:len(y_train)]
X_meta_val = X_meta_full[len(y_train):]

C:\Users\moksh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
from collections import Counter
print("Train labels:", Counter(y_train))
print("Val labels:", Counter(y_val))

Train labels: Counter({3: 1342, 0: 1342, 2: 1342, 4: 1342, 1: 1342})
Val labels: Counter({4: 336, 1: 336, 0: 336, 2: 336, 3: 336})


In [6]:
print("Metadata shape:", X_meta_train.shape)
print("Metadata sample:\n", X_meta_train[2])
print("Metadata variance:", np.var(X_meta_train))

Metadata shape: (6710, 500)
Metadata sample:
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0

In [ ]:
# VGG16 Image-Only Training with Hyperparameter Tuning and Fine-Tuning

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K

# === BASE PATH ===
BASE_PATH = r"C:\\Users\\moksh\\OneDrive\\Documents\\OneDrive\\Desktop\\LCIT\\CV\\Assignment_1"

# === LOAD DATA ===
X_img_train = np.load(os.path.join(BASE_PATH, "X_img_train_bal_q4.npy"), allow_pickle=True)
X_img_val = np.load(os.path.join(BASE_PATH, "X_img_val_bal_q4.npy"), allow_pickle=True)
y_train = np.load(os.path.join(BASE_PATH, "y_train_bal_q4.npy"))
y_val = np.load(os.path.join(BASE_PATH, "y_val_bal_q4.npy"))

# === IMAGE PREPROCESSING ===
def preprocess_image(path, target_size=(224, 224)):
    image = cv2.imread(path)
    if image is None:
        return np.zeros((*target_size, 3))
    image = cv2.resize(image, target_size)
    image = cv2.GaussianBlur(image, (3, 3), 0)
    yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
    image = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return image.astype(np.float32) / 255.0

# === IMAGE-ONLY GENERATOR ===
class ImageOnlyGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size=32, target_size=(224,224), augment=False):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.augment = augment

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, idx):
        idxs = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        batch_imgs = [self.load_and_preprocess(p) for p in self.image_paths[idxs]]
        batch_labels = self.labels[idxs]
        return np.array(batch_imgs, dtype=np.float32), np.array(batch_labels, dtype=np.int32)

    def load_and_preprocess(self, path):
        image = preprocess_image(path, self.target_size)
        if self.augment and np.random.rand() < 0.5:
            image = np.fliplr(image)
        return image

# === MODEL BUILDER (IMAGE ONLY) ===
def build_image_only_model(dense_units=256, dropout_rate=0.3, learning_rate=1e-4):
    image_input = Input(shape=(224, 224, 3), name='image_input')
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=image_input)
    for layer in base_model.layers:
        layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(5, activation='softmax')(x)

    model = Model(inputs=image_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# === GRID SEARCH ===
param_grid = [
    {'dense_units': 128, 'dropout_rate': 0.3, 'learning_rate': 1e-3},
    {'dense_units': 256, 'dropout_rate': 0.3, 'learning_rate': 1e-4},
    {'dense_units': 512, 'dropout_rate': 0.3, 'learning_rate': 5e-5}
]

train_gen = ImageOnlyGenerator(X_img_train, y_train, augment=True)
val_gen = ImageOnlyGenerator(X_img_val, y_val, augment=False)

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
best_model = None
best_val_acc = 0
best_config = None

for i, params in enumerate(param_grid, 1):
    K.clear_session()
    print(f"\n🔧 Running config {i}: {params}")
    model = build_image_only_model(**params)
    history = model.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[early_stop], verbose=1)
    val_acc = history.history['val_accuracy'][-1]
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = model
        best_config = params
    print(f"✅ Val Acc: {val_acc:.4f}")

# === FINE-TUNING BEST MODEL ===
print(f"\n Best config: {best_config}, Val Acc: {best_val_acc:.4f}")

for layer in best_model.layers:
    if hasattr(layer, "name") and layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True

best_model.compile(optimizer=Adam(learning_rate=1e-5),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

fine_tune_history = best_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2),
        ModelCheckpoint(os.path.join(BASE_PATH, 'vgg_imgonly_finetuned.keras'), save_best_only=True)
    ]
)

print(" Fine-Tuning Complete!")




🔧 Running config 1: {'dense_units': 128, 'dropout_rate': 0.3, 'learning_rate': 0.001}


C:\Users\moksh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
210/210 ━━━━━━━━━━━━━━━━━━━━ 907s 4s/step - accuracy: 0.1869 - loss: 1.6622 - val_accuracy: 0.2000 - val_loss: 1.6099
Epoch 2/5
 26/210 ━━━━━━━━━━━━━━━━━━━━ 10:45 4s/step - accuracy: 0.2213 - loss: 1.6106